In [1]:
import pandas as pd
import altair as alt

## Data Cleaning

In [5]:
# global_cancer_predictions

gcp_path = '../data/raw/global_cancer_predictions.csv'
cancer_predictions_data = pd.read_csv(gcp_path)
cancer_predictions_data.head()

,Country,Age_Group,Cancer_Type,Risk_Factors,Incidence,Mortality,Prevalence,Urban_Population,Health_Expenditure_%GDP,Tobacco_Use_%,...,Air_Quality_Index,UV_Radiation,Family_History_%,Genetic_Mutation_%,Treatment_Coverage_%,GDP_per_Capita,Life_Expectancy,Health_Infrastructure_Index,Education_Index,Population_Density
0,Turkey,15-24,Lung,Obesity,44,457,955,32.906758,11.834005,12.578421,...,96,2.877395,44.300862,6.924822,97.210912,29779,82.366306,2.324139,0.875452,736.609006
1,Canada,0-14,Prostate,Genetic,643,278,150,40.207750,6.412955,25.120870,...,69,9.851073,17.727832,7.480867,98.658890,27570,56.791854,2.875398,0.730587,827.101915
2,China,15-24,Breast,Pollution,565,161,1428,62.225708,7.066045,33.662102,...,10,3.156956,45.250388,17.109909,80.144866,50479,62.069261,1.502259,0.757807,139.180394
3,India,15-24,Leukemia,Inactivity,509,117,1996,84.119599,12.102488,29.599358,...,179,6.364374,18.079635,5.280028,84.765434,67829,60.673865,8.945143,0.805388,776.916328
4,Nigeria,15-24,Prostate,Pollution,288,170,383,37.403640,14.487316,15.348235,...,151,9.038135,20.292538,7.763366,81.781825,77986,66.400764,7.332599,0.748743,494.974242


In [8]:
# NHA indicators

nha_path = '../data/raw/NHA_indicators.csv'
nha_data = pd.read_csv(nha_path)
nha_data.head()

,Countries,Indicators,2020,2021,2022
0,Algeria,Current Health Expenditure (CHE) per Capita in...,211,209,180
1,Algeria,Current Health Expenditure (CHE),"9,296","9,334","8,172"
2,Algeria,Out-of-pocket (OOPS) as % of Current Health Ex...,40,40,50
3,Algeria,Out-of-Pocket Expenditure (OOPS) per Capita in...,84,84,90
4,Angola,Current Health Expenditure (CHE) per Capita in...,56,67,101


In [6]:
cancer_predictions_data.shape

(160000, 23)

In [10]:
nha_data.shape

(854, 5)

In [13]:
cancer_predictions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Country                      160000 non-null  object 
 1   Age_Group                    160000 non-null  object 
 2   Cancer_Type                  160000 non-null  object 
 3   Risk_Factors                 160000 non-null  object 
 4   Incidence                    160000 non-null  int64  
 5   Mortality                    160000 non-null  int64  
 6   Prevalence                   160000 non-null  int64  
 7   Urban_Population             160000 non-null  float64
 8   Health_Expenditure_%GDP      160000 non-null  float64
 9   Tobacco_Use_%                160000 non-null  float64
 10  Alcohol_Consumption_Liters   160000 non-null  float64
 11  Physical_Activity_%          160000 non-null  float64
 12  Obesity_%                    160000 non-null  float64
 13 

In [11]:
# Drop NA

cancer_predictions_data = cancer_predictions_data.dropna()
nha_data = nha_data.dropna()

In [33]:
# Data Wrangling for NHA

nha_melted = nha_data.melt(
    id_vars=["Countries", "Indicators"],
    value_vars=["2020", "2021", "2022"],
    var_name="Year",
    value_name="Value"
)

nha_melted = nha_melted.rename(columns={"Countries": "Country"})

nha_pivot = nha_melted.pivot(
    index=["Country", "Year"],
    columns="Indicators",
    values="Value"
).reset_index()

nha_pivot.columns.name = None
nha_pivot.head()

,Country,Year,Current Health Expenditure (CHE),Current Health Expenditure (CHE) per Capita in US$,Out-of-Pocket Expenditure (OOPS) per Capita in US$,Out-of-pocket (OOPS) as % of Current Health Expenditure (CHE),Primary Health Care (PHC) Expenditure per Capita in US$
0,Afghanistan,2020,"3,129",80,60,75,46
1,Afghanistan,2021,"3,261",82,63,77,48
2,Afghanistan,2022,"3,273",81,63,78,46
3,Albania,2020,"1,138",396,235,59,NaN
4,Albania,2021,"1,327",466,278,60,NaN


In [34]:
nha_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 7 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   Country                                                        579 non-null    object
 1   Year                                                           579 non-null    object
 2   Current Health Expenditure (CHE)                               576 non-null    object
 3   Current Health Expenditure (CHE) per Capita in US$             576 non-null    object
 4   Out-of-Pocket Expenditure (OOPS) per Capita in US$             576 non-null    object
 5   Out-of-pocket (OOPS) as % of Current Health Expenditure (CHE)  579 non-null    object
 6   Primary Health Care (PHC) Expenditure per Capita in US$        132 non-null    object
dtypes: object(7)
memory usage: 31.8+ KB


In [35]:
cols_to_convert = nha_pivot.columns.difference(['Country', 'Year'])

for col in cols_to_convert:
    nha_pivot[col] = (
        nha_pivot[col]
        .replace(',', '', regex=True)  
        .replace(' ', '', regex=True) 
        .replace('…', pd.NA, regex=False)
        .replace('', pd.NA)        
    )
    nha_pivot[col] = pd.to_numeric(nha_pivot[col], errors='coerce')

In [36]:
nha_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 7 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Country                                                        579 non-null    object 
 1   Year                                                           579 non-null    object 
 2   Current Health Expenditure (CHE)                               576 non-null    float64
 3   Current Health Expenditure (CHE) per Capita in US$             576 non-null    float64
 4   Out-of-Pocket Expenditure (OOPS) per Capita in US$             576 non-null    float64
 5   Out-of-pocket (OOPS) as % of Current Health Expenditure (CHE)  579 non-null    int64  
 6   Primary Health Care (PHC) Expenditure per Capita in US$        132 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 31.8+ KB


In [37]:
nha_clean = nha_pivot.dropna()
nha_clean.head()

,Country,Year,Current Health Expenditure (CHE),Current Health Expenditure (CHE) per Capita in US$,Out-of-Pocket Expenditure (OOPS) per Capita in US$,Out-of-pocket (OOPS) as % of Current Health Expenditure (CHE),Primary Health Care (PHC) Expenditure per Capita in US$
0,Afghanistan,2020,3129.0,80.0,60.0,75,46.0
1,Afghanistan,2021,3261.0,82.0,63.0,77,48.0
2,Afghanistan,2022,3273.0,81.0,63.0,78,46.0
21,Armenia,2020,1547.0,535.0,417.0,78,249.0
22,Armenia,2021,1710.0,596.0,469.0,79,285.0


In [39]:
nha_avg = nha_clean.groupby("Country").mean(numeric_only=True).round(2).reset_index()
nha_avg.head()

,Country,Current Health Expenditure (CHE),Current Health Expenditure (CHE) per Capita in US$,Out-of-Pocket Expenditure (OOPS) per Capita in US$,Out-of-pocket (OOPS) as % of Current Health Expenditure (CHE),Primary Health Care (PHC) Expenditure per Capita in US$
0,Afghanistan,3221.00,81.00,62.00,76.67,46.67
1,Armenia,1733.67,602.00,473.33,78.67,265.67
2,Austria,53340.33,5967.67,957.67,16.00,2311.33
3,Belarus,4420.67,477.67,118.67,25.00,174.33
4,Belgium,62812.67,5431.33,1015.33,18.67,1984.67


In [42]:
nha_avg.shape

(44, 6)

In [45]:
cancer_country_avg = (
    cancer_predictions_data
    .groupby("Country")
    .mean(numeric_only=True)
    .round(2)
    .reset_index()
)
cancer_country_avg.head()

,Country,Incidence,Mortality,Prevalence,Urban_Population,Health_Expenditure_%GDP,Tobacco_Use_%,Alcohol_Consumption_Liters,Physical_Activity_%,Obesity_%,Air_Quality_Index,UV_Radiation,Family_History_%,Genetic_Mutation_%,Treatment_Coverage_%,GDP_per_Capita,Life_Expectancy,Health_Infrastructure_Index,Education_Index,Population_Density
0,Afghanistan,507.64,253.58,1036.56,60.26,8.99,24.81,7.98,50.08,25.04,99.42,5.99,30.24,12.49,75.49,39853.39,67.52,5.50,0.65,503.18
1,Angola,501.10,252.01,1039.06,60.54,9.01,24.93,8.12,50.45,25.23,100.61,6.06,29.85,12.40,75.11,39928.05,67.54,5.53,0.65,502.85
2,Argentina,499.29,252.59,1049.86,59.95,8.92,25.04,7.98,49.57,25.11,99.66,6.08,30.10,12.41,74.86,40746.75,67.40,5.52,0.65,508.14
3,Australia,491.35,252.26,1057.39,59.58,8.98,25.09,8.12,49.74,24.66,99.86,5.99,30.12,12.56,75.02,41080.27,67.42,5.55,0.65,510.96
4,Bangladesh,498.33,247.76,1053.07,60.05,8.98,25.23,8.01,50.10,25.12,99.83,6.00,30.01,12.40,75.31,40634.31,67.17,5.49,0.65,509.01


In [53]:
cancer_country_avg.shape

(51, 20)

In [54]:

print(sorted(cancer_country_avg["Country"].unique()))


['Afghanistan', 'Angola', 'Argentina', 'Australia', 'Bangladesh', 'Brazil', 'Cameroon', 'Canada', 'China', 'Colombia', 'Democratic Republic of the Congo', 'Egypt, Arab Rep.', 'Ethiopia', 'France', 'Germany', 'Ghana', 'India', 'Indonesia', 'Iran (Islamic Republic of)', 'Iraq', 'Italy', 'Japan', 'Madagascar', 'Malaysia', 'Mexico', 'Morocco', 'Mozambique', 'Nepal', 'Niger', 'Nigeria', 'Pakistan', 'Peru', 'Philippines', 'Poland', 'Republic of Korea', 'Russian Federation', 'Saudi Arabia', 'South Africa', 'Spain', 'Sri Lanka', 'Sudan', 'Thailand', 'Turkey', 'Uganda', 'Ukraine', 'United Kingdom of Great Britain and Northern Ireland', 'United States of America', 'Uzbekistan', 'Venezuela, RB', 'Viet Nam', 'Yemen']


In [55]:

print(sorted(nha_avg["Country"].unique()))


['Afghanistan', 'Armenia', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Burkina Faso', 'Canada', 'Chad', 'Costa Rica', 'Croatia', 'Cyprus', 'Czechia', 'Democratic Republic of the Congo', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Ghana', 'Hungary', 'Iceland', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Mexico', 'Mongolia', 'Myanmar', 'Netherlands (Kingdom of the)', 'Niger', 'North Macedonia', 'Norway', 'Paraguay', 'Poland', 'Republic of Moldova', 'Romania', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom of Great Britain and Northern Ireland', 'Uruguay', 'occupied Palestinian territory, including east Jerusalem']
